<a ><img src="Pictures/trashpanda.png"  width="70" align="left"> </a>
<div style="text-align: right"> <h3><span style="color:gray"> DRAFT </span> </h3> </div>

<br>
<br>
<br>

<a ><img src="Pictures/Icon.png" Width="250" align="center"> </a>

<br>
<br>



<h1><center>Sentiment Analysis</center></h1>
<h2><center> <span style="font-weight:normal"><font color='#022F73'> 
Natural Language Processing (NLP): <br> Analysis of Headlines with VADER and pysentiment </font>  </span></center></h2>


<h3><center><font color='gray'>JONAS GOTTAL</font></center></h3>

#### About this Notebook
We will conduct a sentiment Analysis with ```VADER``` *(Valence Aware Dictionary for Sentiment Reasoning)*  <br> 
and ```pysentiment```, a pre-trained finance-focused Model, on headlines of several different news outlets with <br> 
impact on german financial markets and generate for each a dedicated sentiment KPI per trading day. <br> 
In the end we will have a sound time series  and - hopefully - some predictive results.<br> 
*The approach is a more explorative data analysis, therefore some parts will be redundant in the end.*
<br>
<br>


<h1>Table of contents</h1>

<div class="alert alert-block alert-info" style="text-decoration:none; margin-top: 30px; background-color:#F2F2F2; border-color:#022F73">
    <span style="color:#022F73">
    <ol>
      <li><a href="#1"> <span style="color:#022F73;text-decoration:underline;text-decoration-color:#F2F2F2" 
       >Motivation  </span> </a></li>
      <li><a href="#2"> <span style="color:#022F73;text-decoration:underline;text-decoration-color:#F2F2F2" 
       >Approach  </span> </a></li>
      <li><a href="#3"> <span style="color:#022F73;text-decoration:underline;text-decoration-color:#F2F2F2" 
       >Sentiment Scores with VADER and pysentiment</span> </a></li>
      <li><a href="#4"> <span style="color:#022F73;text-decoration:underline;text-decoration-color:#F2F2F2" 
       >Upload on our MySQL Database</span> </a></li>
      <li><a href="#5"> <span style="color:#022F73;text-decoration:underline;text-decoration-color:#F2F2F2" 
       >Schedule daily runs as .py script</span> </a></li>
      <li><a href="#6"> <span style="color:#022F73;text-decoration:underline;text-decoration-color:#F2F2F2" 
       >Analysis of our unique dataset</span> </a></li>
      <li><a href="#7"> <span style="color:#022F73;text-decoration:underline;text-decoration-color:#F2F2F2" 
       >Outlook</span> </a></li>
    </ol>
    </span>
</div>

In [1]:
#!pip install html5lib
#!pip install ipython-sql
#!pip install nltk
#! pip install pysentiment2

import requests
from   bs4 import BeautifulSoup
import html5lib
import time
import numpy as np
import pandas as pd
import matplotlib as mp
import matplotlib.pyplot as plt
import dtale
import sqlalchemy
from sqlalchemy import create_engine
import pymysql
pymysql.install_as_MySQLdb()
import pysentiment2 as ps
import nltk
import re
from collections import Counter
from nltk.sentiment.vader import SentimentIntensityAnalyzer as SIA
import seaborn as sns

# 1. Motivation <a id="1"></a>


> <i> "There are three ways to make a living in this business: be first, be smarter, or cheat. Well, I don't cheat. And although I like to think we have some pretty smart people here in this room, it sure is a hell of lot easier to just be first." </i> **- Margin Call** 

But to be the first to use market data, you would need to be in an optimal physical location (depending on the stock exchanges) and you have to have the fastest connection. Neither of those is true for me and many others. So to be successful - in this analogy of margin call - you would have to design/manipulate a unique set of data - and therefore be first again.

## Why headlines of causal News Outlets?

The market participants have drastically changed in the last years: The most influencing Trades are from the retail traders! 
(under the assumption that HF Traders don't change prices significantly and only profit from small fractions and arbitrage opportunities)

<a ><img src="Pictures/Retail1.png"  width="600" align="left"> </a>



<a ><img src="Pictures/Retail2.png"  width="350" align="right"> </a>


And therefore we focus on the source of information of those retail traders: news outlets

And because most only skim the contents and mostly only read the headline, we will focus on their sentiment.


Why not use NLP on specific stocks (e.g. only news articles on AAPL )?
My assumption is, that no (retail) investor will make a decision on one single (normal) news headline.
But the entirety of the current market sentiment, might be a predictor of market activity and volatility and hence daily risk appetite. Therefore we are going to analyse the total addressable market of media outlets. 

(Also there are already too many (large) market participants who analyse the sentiment by single stocks for news articles and social media/forum posts {e.g. r/wsb}— so not an opportunity for strat arb!)

## Revision of our news scraping approach (see other notebook before)

#### Documentation: https://requests.readthedocs.io/en/master/user/advanced
```j
content = requests.get(url = url, headers = {'Accept': 'text/html'}, timeout=(3, 27))
`````


### A. Bild

In [2]:
url = "https://www.bild.de/politik/international/bild-international/home-44225950.bild.html"

content = requests.get(url = url, headers = {'Accept': 'text/html'}, timeout=(3, 27))

content.status_code

200

In [3]:
bild = BeautifulSoup(content.content, 'html.parser')

headlines = bild.find_all('h3')

df_bild = pd.DataFrame([i.text for i in headlines], columns=["bild"]).replace("\n", "", regex=True).replace("\t", "", regex=True)
print (df_bild.size, df_bild.head())

12                                                 bild
0                  Bill Gates on conspiracy theories
1                    We have watched this war happen
2  „The city looks like German cities during the ...
3                   German troops pull out of Kunduz
4               “Run! The planes are bombing again!”


### B. Spiegel

In [4]:
url = "https://www.spiegel.de/international/"

content = requests.get(url = url, headers = {'Accept': 'text/html'}, timeout=(3, 27))

content.status_code

200

In [5]:
spiegel = BeautifulSoup(content.content, 'html.parser')

headlines = spiegel.find_all(attrs={'class':'align-middle'})

df_spiegel = pd.DataFrame([i.text for i in headlines], columns=["spiegel"])
print (df_spiegel.size, df_spiegel.head())

28                                              spiegel
0                        Where Did America Go Wrong?
1         "The World Could Have Learned from Africa"
2  Nagorno-Karabakh Settles in for Five Years of ...
3  Geopolitics of COVID: The Unprecedented Challe...
4    "I'm Horrified By What Is Happening in Belarus"


### C. Zeit

In [6]:
url = "https://www.zeit.de/english/index"

content = requests.get(url = url, headers = {'Accept': 'text/html'}, timeout=(3, 27))

content.status_code

200

In [7]:
zeit = BeautifulSoup(content.content, 'html.parser')

headlines = zeit.find_all(attrs={'class':'zon-teaser-standard__title'})

df_zeit = pd.DataFrame([i.text for i in headlines], columns=["zeit"])
print (df_zeit.size, df_zeit.head())

20                                                 zeit
0  Why Is the Risk of Coronavirus Transmission so...
1          "We Have Always Stigmatized Single Women"
2                                 The Irreconcilable
3                 "Trump is only performing fascism"
4                 How To Protect Yourself and Others


### D. FAZ

In [8]:
url = "https://www.faz.net/english/"

content = requests.get(url = url, headers = {'Accept': 'text/html'}, timeout=(3, 27))

content.status_code

200

In [9]:
faz = BeautifulSoup(content.content, 'html.parser')

headlines = faz.find_all(attrs={'class':'tsr-Base_HeadlineText'})

df_faz = pd.DataFrame([i.text for i in headlines], columns=["faz"]).replace("\n", "", regex=True).replace("\t", "", regex=True)
print (df_faz.size, df_faz.head())

20                                                  faz
0  „We have to fight misinformation because peopl...
1                  Do not ruin the Internet!        
2                              The Black Axe        
3  Time for a German-British Friendship Treaty   ...
4                 The Girl With The Red Coat        


### E. BBC

In [10]:
url = "https://www.bbc.com/news"

content = requests.get(url = url, headers = {'Accept': 'text/html'}, timeout=(3, 27))

content.status_code

200

In [11]:
bbc = BeautifulSoup(content.content, 'html.parser')

headlines = bbc.find_all('h3')

df_bbc = pd.DataFrame([i.text for i in headlines], columns=["bbc"])
print (df_bbc.size, df_bbc.head())

54                                                  bbc
0  Trump signals he is prepared to leave White House
1  Trump signals he is prepared to leave White House
2   'A genius': Huge crowds bid farewell to Maradona
3        'This is War': Poland’s battle for abortion
4  Politicians throw pig guts at each other in Ta...


### F. CNN

In [12]:
url = "https://www.cnn.com/business"

content = requests.get(url = url, headers = {'Accept': 'text/html'}, timeout=(3, 27))

content.status_code

200

In [13]:
cnn = BeautifulSoup(content.content, 'html.parser')

headlines = cnn.find_all(attrs={'class': 'cd__headline'})

df_cnn = pd.DataFrame([i.text for i in headlines], columns=["cnn"])
print (df_cnn.size, df_cnn.head())

33                                                  cnn
0  China slaps tariffs of up to 212% on Australia...
1  A growing number of food brands want you to kn...
2  Knife-edge Swiss vote could make businesses li...
3  Opinion McDonald's CEO: We're thinking differe...
4  'Tired of being sad.' The financial stress fro...


### G. CNBC

In [14]:
url = "https://www.cnbc.com"

content = requests.get(url = url, headers = {'Accept': 'text/html'}, timeout=(3, 27))

content.status_code

200

In [15]:
cnbc = BeautifulSoup(content.content, 'html.parser')

headlines = cnbc.find_all(attrs={'class':'Card-title'})

df_cnbc = pd.DataFrame([i.text for i in headlines], columns=["cnbc"])
print (df_cnbc.size, df_cnbc.head())

34                                                 cnbc
0  U.S. stock futures edge higher as investors mo...
1        Treasury yields edge lower amid low volumes
2  After market's November surge, there may be le...
3  Fed weighed adjusting bond purchases to provid...
4  Britain tries to cool AstraZeneca concerns as ...


### H. New York Times

Due to its sound reputation we will also use the sub sections and not only the main site.

For us relevant: 
- World https://www.nytimes.com/section/world
- Politics https://www.nytimes.com/section/politics
- Business https://www.nytimes.com/section/business
- Tech https://www.nytimes.com/section/technology

In [16]:
url = "https://www.nytimes.com/section/world"

content = requests.get(url = url, headers = {'Accept': 'text/html'}, timeout=(3, 27))

content.status_code

200

In [17]:
nyt_w = BeautifulSoup(content.content, 'html.parser')

headlines = nyt_w.find_all(name="h2")

df_nyt_w = pd.DataFrame([i.text for i in headlines], columns=["nyt_w"]).iloc[1:].reset_index(drop=True)
print (df_nyt_w.size, df_nyt_w.head())

44                                                nyt_w
0  Argentina, and Latin America, Mourn Maradona a...
1  A Soaring Monument to Beauty in China Is Stirr...
2  Australian Military Moves to Dismiss Soldiers ...
3  ‘The Crown’ Stokes an Uproar Over Fact vs. Ent...
4  E.U. Border Agency Accused of Covering Up Migr...


In [18]:
url = "https://www.nytimes.com/section/politics"

content = requests.get(url = url, headers = {'Accept': 'text/html'}, timeout=(3, 27))

content.status_code

200

In [19]:
nyt_p = BeautifulSoup(content.content, 'html.parser')

headlines = nyt_p.find_all(name="h2")

df_nyt_p = pd.DataFrame([i.text for i in headlines], columns=["nyt_p"]).iloc[1:].reset_index(drop=True)
print (df_nyt_p.size, df_nyt_p.head())

18                                                nyt_p
0  ‘Loser’: How a Lifelong Fear Bookended Trump’s...
1  Military’s Role in Vaccine Will Be Strictly Be...
2  Trump, Still Claiming Victory, Says He Will Le...
3  Senate Democrats Face Power Struggle for Top J...
4  Trump, Still Claiming Victory, Says He Will Le...


In [20]:
url = "https://www.nytimes.com/section/business"

content = requests.get(url = url, headers = {'Accept': 'text/html'}, timeout=(3, 27))

content.status_code

200

In [21]:
nyt_b = BeautifulSoup(content.content, 'html.parser')

headlines = nyt_b.find_all(name="h2")

df_nyt_b = pd.DataFrame([i.text for i in headlines], columns=["nyt_b"]).iloc[1:].reset_index(drop=True)
print (df_nyt_b.size, df_nyt_b.head())

35                                                nyt_b
0  Pushed by Pandemic, Amazon Goes on a Hiring Sp...
1  Brexit Is Nipping at London’s Role as a Financ...
2  Black Friday Live Updates: Holiday Shopping in...
3  ‘Tokenized’: Inside Black Workers’ Struggles a...
4  This Year’s Black Friday Designer Fashion Sale...


In [22]:
url = "https://www.nytimes.com/section/technology"

content = requests.get(url = url, headers = {'Accept': 'text/html'}, timeout=(3, 27))

content.status_code

200

In [23]:
nyt_t = BeautifulSoup(content.content, 'html.parser')

headlines = nyt_t.find_all(name="h2")

df_nyt_t = pd.DataFrame([i.text for i in headlines], columns=["nyt_t"]).iloc[1:].reset_index(drop=True)
print (df_nyt_t.size, df_nyt_t.head())

23                                                nyt_t
0  Patients of a Vermont Hospital Are Left ‘in th...
1  It’s Time for a Digital Detox. (You Know You N...
2  Facebook Struggles to Balance Civility and Growth
3  Meet GPT-3. It Has Learned to Code (and Blog a...
4                                Personal Technology


# 2. Approach <a id="2"></a>

> <i>"Natural language processing (NLP) is a subfield of linguistics, computer science, and artificial intelligence concerned with the interactions between computers and human language, in particular how to program computers to process and analyze large amounts of natural language data." </i> - Wikipedia

Sentiment analysis is a text analysis method that detects polarity (e.g. a positive or negative opinion) within the text, whether a whole document, paragraph, sentence, or clause.

Sentiment analysis aims to measure the attitude, sentiments, evaluations, attitudes, and emotions of a speaker/writer based on the computational treatment of subjectivity in a text.

## ```VADER```

VADER ( Valence Aware Dictionary for Sentiment Reasoning) is a model used for text sentiment analysis that is sensitive to both polarity (positive/negative) and intensity (strength) of emotion. It is available in the ```NLTK``` package and can be applied directly to unlabeled text data.

VADER sentimental analysis relies on a dictionary that maps lexical features to emotion intensities known as sentiment scores. The sentiment score of a text can be obtained by summing up the intensity of each word in the text.

For example- Words like ‘love’, ‘enjoy’, ‘happy’, ‘like’ all convey a positive sentiment. Also VADER is intelligent enough to understand the basic context of these words, such as “did not love” as a negative statement. It also understands the emphasis of capitalization and punctuation, such as “ENJOY”

### Polarity classification

We won’t try to determine if a sentence is objective or subjective, fact or opinion. Rather, we care only if the text expresses a positive, negative or neutral opinion.

## ```pysentiment```

This is a library for sentiment analysis in dictionary framework. Two dictionaries are provided in the library, namely, *[Harvard IV-4](http://www.wjh.harvard.edu/~inquirer/homecat.htm)* and *[Loughran and McDonald Financial Sentiment Dictionaries](https://sraf.nd.edu/)*, which are sentiment dictionaries for general and financial sentiment analysis.

##### Introduction

```Positive``` and ```Negative``` are word counts for the words in positive and negative sets.

```Polarity``` and ```Subjectivity``` are calculated in the same way of [Lydia](http://www.cs.sunysb.edu/~skiena/lydia/) system.

The formula for Polarity is:

$$
Polarity = \frac{Pos - Neg}{Pos + Neg}
$$

The formula for Subjectivity is:

$$
Subjectivity = \frac{Pos + Neg}{count}
$$


# 3. Sentiment Scores  <a id="3"></a>

## ... with VADER <a id="3"></a>

Let's have a look at our polarity scores with VADER!

In [24]:
nltk.download("vader_lexicon")

def get_score(row):
    sid=SIA()
    score=sid.polarity_scores(row)
    return score["compound"]

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/jonas/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [25]:
df_bild_score = df_bild["bild"].apply(get_score)
df_bild_final = pd.concat([df_bild, df_bild_score ], axis=1, sort=False).rename(columns={"bild": "bild", "bild": "score"}, errors="raise")

In [26]:
df_bild_final.head(10)

,score,score
0,Bill Gates on conspiracy theories,-0.5267
1,We have watched this war happen,-0.6833
2,„The city looks like German cities during the ...,0.3612
3,German troops pull out of Kunduz,0.0000
4,“Run! The planes are bombing again!”,0.0000
5,Church bombed to ruins,-0.4404
6,Navalny: Sanction Putin's closest circle,0.0000
7,"China is celebrating,the world is suffering",-0.4767
8,Why the US is withdrawing from the Open Skies ...,0.0000
9,The Thai king’s secret torture prison,-0.8020


Not surprising, that **torture**, **bombed to ruins** and **conspiracy theories** don't have the most positive connotation. 

But because the number of Articles differ from news outlet to news outlet, we will only include the more robust mean of our scores:

In [27]:
daily_bild = df_bild_final["score"].mean()
daily_bild

score   -0.213992
dtype: float64

In [28]:
#To wrap it up:
bild_score = df_bild["bild"].apply(get_score)
daily_bild = bild_score.mean()
daily_bild

-0.21399166666666666

In [29]:
#Now for all:
news = ["bild", "spiegel", "zeit", "faz", "bbc", "cnn", "cnbc", "nyt_w", "nyt_p", "nyt_b", "nyt_t"]
for k in news:
    exec(f'{k}_score = df_{k}["{k}"].apply(get_score)')
    exec(f'vader_{k} = {k}_score.mean()') 

## with [```pysentiment``` ](https://nickderobertis.github.io/pysentiment/)

We have two different dictionaries, namely *[Harvard IV-4](http://www.wjh.harvard.edu/~inquirer/homecat.htm)* and *[Loughran and McDonald Financial Sentiment Dictionaries](https://sraf.nd.edu/)*. Therefore we have two different *models*:

In [30]:
hiv4 = ps.HIV4()
lm = ps.LM()

Next we define our ```pyscore``` 

In [31]:
def get_pyscore(row,model):
    return pd.Series(model.get_score(model.tokenize(text=row)))

Let's use our example with Bild again:

In [32]:
bild_score_h = df_bild["bild"].apply(get_pyscore, model=hiv4)
bild_score_h.columns = ['Positive_h', 'Negative_h', 'Polarity_h', 'Subjectivity_h']
bild_score_l = df_bild["bild"].apply(get_pyscore, model=lm)
bild_score_l.columns = ['Positive_l', 'Negative_l', 'Polarity_l', 'Subjectivity_l']

bild_scores=pd.concat([bild_score_h, bild_score_l], axis=1)

In [33]:
bild_scores

,Positive_h,Negative_h,Polarity_h,Subjectivity_h,Positive_l,Negative_l,Polarity_l,Subjectivity_l
0,0.0,1.0,-0.999999,0.333333,0.0,1.0,-0.999999,0.333333
1,0.0,1.0,-0.999999,0.333333,0.0,0.0,0.000000,0.000000
2,2.0,1.0,0.333333,0.750000,0.0,0.0,0.000000,0.000000
3,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000
4,0.0,2.0,-1.000000,0.666666,0.0,0.0,0.000000,0.000000
5,0.0,2.0,-1.000000,1.000000,0.0,0.0,0.000000,0.000000
6,0.0,1.0,-0.999999,0.166667,0.0,0.0,0.000000,0.000000
7,2.0,1.0,0.333333,1.000000,0.0,1.0,-0.999999,0.333333
8,2.0,0.0,1.000000,0.500000,0.0,0.0,0.000000,0.000000
9,0.0,3.0,-1.000000,0.750000,0.0,0.0,0.000000,0.000000


The results seem a bit more binary in comparison to our VADER results
Also we should drop the columns *Positive* and *Negative*, because they don't have a lot of meaning to us - the polarity and subjectivity scores are aggregated from them.

In [34]:
bild_score_h = df_bild["bild"].apply(get_pyscore, model=hiv4).drop(['Positive', 'Negative'], axis=1)
bild_score_h.columns = ['Polarity_h', 'Subjectivity_h']

bild_score_l = df_bild["bild"].apply(get_pyscore, model=lm).drop(['Positive', 'Negative'], axis=1)
bild_score_l.columns = [ 'Polarity_l', 'Subjectivity_l']

In [35]:
bild_scores = pd.concat([bild_score_h, bild_score_l], axis=1)

In [36]:
bild_scores =  (bild_scores.mean().to_frame().T)

In [37]:
bild_scores.insert(loc=0, column='bild', value=daily_bild)


In [38]:
bild_scores

,bild,Polarity_h,Subjectivity_h,Polarity_l,Subjectivity_l
0,-0.213992,-0.444444,0.569444,-0.25,0.097222


Now a bit more compact: 

In [39]:
score_h = df_bild["bild"].apply(get_pyscore, model=hiv4).drop(['Positive', 'Negative'], axis=1)
score_h.columns = ['Polarity_h', 'Subjectivity_h']

score_l = df_bild["bild"].apply(get_pyscore, model=lm).drop(['Positive', 'Negative'], axis=1)
score_l.columns = [ 'Polarity_l', 'Subjectivity_l']

scores = pd.concat([score_h, score_l], axis=1)
scores = scores.mean().to_frame().T
scores.insert(loc=0, column='bild', value=daily_bild)
bild_scores = scores

In [40]:
bild_scores

,bild,Polarity_h,Subjectivity_h,Polarity_l,Subjectivity_l
0,-0.213992,-0.444444,0.569444,-0.25,0.097222


Now all the news outlets, lets start with **HIV4**

In [41]:
for k in news:
    exec(f'{k}_score_HV = df_{k}["{k}"].apply(get_pyscore, model=hiv4).drop(["Positive", "Negative"], axis=1)') 
    exec(f'{k}_score_HV.columns = ["polarity_hv_{k}", "subjectivity_hv_{k}"]')

Now the **LM** model

In [42]:
for k in news:
    exec(f'{k}_score_LM = df_{k}["{k}"].apply(get_pyscore, model=lm).drop(["Positive", "Negative"], axis=1)') 
    exec(f'{k}_score_LM.columns = ["polarity_lm_{k}", "subjectivity_lm_{k}"]') 

Now lets put it together and calculate the mean

In [43]:
for k in news:
    exec(f'{k}_scores = pd.concat([{k}_score_HV, {k}_score_LM], axis=1)') 
    exec(f'{k}_scores = {k}_scores.mean().to_frame().T')

# 4. Upload on our MySQL Database <a id="4"></a>

Let's finally merge our results from our sentiment analysis and add a timestamp

In [44]:
for k in news:
    exec(f'{k}_scores.insert(loc=0, column="vader_{k}", value = vader_{k})')

Lets store it in a Data Frame

In [45]:
daily_scores = pd.DataFrame()
for k in news:
    exec(f'daily_scores = pd.concat([daily_scores, {k}_scores], axis=1, sort=False)')

##### Lets add a Timestamp so we can use the Headlines as Timeseries

In [46]:
n = pd.to_datetime('today').strftime("%m/%d/%Y")
n

'11/27/2020'

In [47]:
daily_scores.insert(loc=0, column='Timestamp', value=n)

In [48]:
daily_scores

,Timestamp,vader_bild,polarity_hv_bild,subjectivity_hv_bild,polarity_lm_bild,subjectivity_lm_bild,vader_spiegel,polarity_hv_spiegel,subjectivity_hv_spiegel,polarity_lm_spiegel,...,vader_nyt_b,polarity_hv_nyt_b,subjectivity_hv_nyt_b,polarity_lm_nyt_b,subjectivity_lm_nyt_b,vader_nyt_t,polarity_hv_nyt_t,subjectivity_hv_nyt_t,polarity_lm_nyt_t,subjectivity_lm_nyt_t
0,11/27/2020,-0.213992,-0.444444,0.569444,-0.25,0.097222,-0.074993,-0.297619,0.356888,-0.285714,...,0.070551,0.161905,0.293605,-0.028571,0.060408,0.010943,0.072464,0.26765,-0.086956,0.079762


In [ ]:
############### CONFIGURE THIS ###################
# Open database connection

# Credentials to database connection
user = 'sql7372795'
passw = 'tbd'
host =  'sql7.freemysqlhosting.net'
database = 'sql7372795'


# Create SQLAlchemy engine to connect to MySQL Database
engine = create_engine("mysql+pymysql://{user}:{pw}@{host}/{db}".format(host=host, db=database, user=user, pw=passw))

In [ ]:
daily_scores.to_sql('scores', engine, if_exists = 'append')

Now we could schedule daily parsing and uploads!

# 5. Schedule daily runs as .py script <a id="5"></a>

##### To create our final database, we will run the crucial parts of our notebook as a .py script 

- HTML Parsing
- Daily ```VADER``` Scores
- Daily ```Pysentiment``` Scores 
- Upload (```append```) to our MySQL Database

All the parts of our exploratory data analysis are obviously not needed for the daily runs.

##### Update
Due to our "free" SQL Database trying to charge money to continue their service and also every "free" server-based script scheduling asked immediately for a credit card number, we run the script daily local on a laptop at the same time (in the morning before markets opened).

# 6. Analysis of our unique dataset <a id="6"></a>

##### Our main goal is to compares our different scores of our timeseries against german financial markets

We will compare:
- ```VADER``` Score
- Polarity and Subjectivity ```Pysentiment HIV-4``` Scores 
- Polarity and Subjectivity ```Pysentiment LM``` Scores 

I am very excited to see our results in a view weeks!

##### Why do we run the script before the markets open? Leading Indicator vs Lagging Indicator
**IF** there is a correlation, we would need to know wether the sentiment influences the markets or the other way around.
If we schedule the script before the markets are open and within our dataset the sentiment proves to be a leading indicator, we can leverage this by running the script as a market indicator verify it by paper trading.

#### Normalized Dataframes for more robust statistics

**Z-score** (a.k.a. *Standard score*) is the number of standard deviations ($\sigma$) by which the value ($x$) of a raw score (i.e., an observed value or data point) is above or below the mean value ($\mu$) of what is being observed or measured.



$$ 
z = \frac{x - \mu }{\sigma} \\
$$

$z$ = z-Score;
$x$ = value;
$\mu$ = mean;
$\sigma$ = standard deviation

# 7. Outlook <a id="7"></a>

The preliminary results of our ```VADER```  and ```pysentiment``` analysis are quite encouraging. But we will have to wait until our dataset has a statistically significant size.

**Preliminary thoughts of critique:** 
- We started our data collection in the midst of one of the most controversial elections (and the markets didn't react as expectad throughout the pandemic)
- The articles didn't change daily for every news outlet - not a reliable leading indicator (maybe social media pages from news outlets are faster?)
- The articles didn't always focus on finance related subjects (maybe only the financial section next time?)
- No parsing for mentioned stocks
- No parsing for user interaction (but do *those* users even participate in financial markets?)

# Thank you for completing this Notebook!

<hr>
<h5>
<p> <center>  <font color='gray'> Jupyter Notebook by </font> Jonas Gottal </center></p>
</h5>